In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import imgaug as ia
from imgaug import augmenters as iaa
from CA1_Yolo import *
from keras.optimizers import Adam
import cv2

In [2]:
img_dir = "../data/image/"
anno_dir = "../data/annotation/"

In [3]:
yoloAPI = YoloV3_API(
    img_dir=img_dir,
    annotation_dir=anno_dir,
    saved_model_name='yolo_model.h5',
    train_size=0.8,
    height=416,
    width=416,
    threshold=0.5,
    batch_size=2)

SyntaxError: invalid syntax (<ipython-input-3-1901c5fd772d>, line 5)

In [4]:
history = yoloAPI.fit_generator(epoch=300, lr=1e-4)


Epoch 1/300
Current Size: 416 x 416
Current Size: 576 x 576
Current Size: 576 x 576
Current Size: 576 x 576
Current Size: 576 x 576
Current Size: 576 x 576
Current Size: 576 x 576
Current Size: 576 x 576
Current Size: 576 x 576
Current Size: 576 x 576
  1/154 [..............................] - ETA: 1:13:24 - loss: 122.6031 - yolo_loss_layer_1_loss: 17.3343 - yolo_loss_layer_2_loss: 35.9658 - yolo_loss_layer_3_loss: 66.9759Current Size: 576 x 576


ResourceExhaustedError: 2 root error(s) found.
  (0) Resource exhausted: OOM when allocating tensor with shape[2,72,72,256] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node batch_normalization_18/cond/Merge-1-1-TransposeNCHWToNHWC-LayoutOptimizer}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[loss/yolo_loss_layer_3_loss/Mean_2/_2935]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

  (1) Resource exhausted: OOM when allocating tensor with shape[2,72,72,256] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node batch_normalization_18/cond/Merge-1-1-TransposeNCHWToNHWC-LayoutOptimizer}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_alloc